# Importación de librerías

In [1]:
#Librerias PI
from PIconnect import PIData, PIServer, PIConfig

# Librerias de Python
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

OSIsoft(r) AF SDK Version: 3.0.2.5


# Funciones para DESCARGA DATOS PI OSISOFT AVEVA

In [2]:
# Configuración del servidor PI
PIServer.DEFAULT_SERVER = 'uwgepi'  # Cambia por el nombre de tu servidor
usuario = 'UF183530'  # Cambia por tu usuario
contraseña = 'UF183530'  # Cambia por tu contraseña
# Configurar la zona horaria predeterminada de PIconnect
PIConfig.DEFAULT_TIMEZONE = 'Europe/Madrid'

In [3]:
# Fecha inicial (yyyy, mm, dd, hh, mm, ss)
fecha_inicio = datetime(2014, 1, 1, 0, 0, 0)  # Cambia por la fecha de inicio deseada
# Fecha fin: se puede poner un delta de tiempo 
# fecha_fin =fecha_inicio + timedelta(days=20)
fecha_fin = datetime(2025, 2, 1, 0, 0, 0)  # Cambia por el rango de tiempo deseado
intervalo = "10s" # Formato PI
# Definicion de tags a descargar
tags = {
    "SAB:CBOP.A81.PAC11.AP001XH01",
    "SAB:CBOP.A81.PAC13.AP001XH01"
}
# Filtro para descarga. Dejar vacio sino se quiere filtro
# Los tags deben ir entre ''
#filtro = "'SAB:G1.TNH_V'>2995" 
filtro = ""

In [ ]:
# Descarga datos de PI en carpeta "data" 
# Crea un fichero por cada mes del año

# Conexión al servidor
with PIServer() as server:
    # Iterar de mes en mes entre las fechas especificadas
    fecha_actual = fecha_inicio
    while fecha_actual < fecha_fin:
        # Calcular el final del mes actual
        mes_siguiente = (fecha_actual.replace(day=28) + timedelta(days=4)).replace(day=1)
        fecha_siguiente = min(mes_siguiente, fecha_fin)
        
        # Crear un DataFrame vacío para el mes actual
        df_mes = pd.DataFrame()
        
        for tag in tags:
            print(f"Descargando datos para {tag} desde {fecha_actual} hasta {fecha_siguiente}...")
            # Buscar la etiqueta en el servidor
            punto = server.search(tag)
            if not punto:
                print(f"Advertencia: No se encontró la etiqueta {tag}")
                continue
            
            punto = punto[0]  # Toma el primer resultado de la búsqueda
            
            # Recuperar datos interpolados para el rango mensual
            valores = punto.interpolated_values(fecha_actual, fecha_siguiente, intervalo, filtro)
            
            # Convertir a un DataFrame temporal para normalizar los datos
            df_temp = pd.DataFrame(valores.items(), columns=["Timestamp", tag])
            df_temp.set_index("Timestamp", inplace=True)
            
            # Unir los datos del tag actual al DataFrame del mes
            if df_mes.empty:
                df_mes = df_temp
            else:
                df_mes = df_mes.join(df_temp, how='outer')
        
        # Crear carpeta para el año si no existe
        year_folder = os.path.join("datos", str(fecha_actual.year))
        os.makedirs(year_folder, exist_ok=True)
        
        # Guardar el DataFrame del mes en un archivo CSV
        csv_filename = os.path.join(year_folder, f"{fecha_actual.year}_{fecha_actual.month:02d}.csv")
        df_mes.to_csv(csv_filename, sep=';', decimal=',')
        
        # Avanzar al siguiente mes
        fecha_actual = fecha_siguiente



# Crear dataframe de todos los CSV descargados

In [7]:
# Función para crear dataframe
def leer_csv_en_carpetas(ruta_base):
    # Crear una lista para almacenar los DataFrames
    dataframes = []
    
    # Recorrer carpetas y subcarpetas con os.walk
    for carpeta, subcarpetas, archivos in os.walk(ruta_base):
        #print(f"Explorando carpeta: {carpeta}")  # Depuración
        for archivo in archivos:
            if archivo.endswith('.csv'):  # Filtrar solo archivos CSV
                ruta_completa = os.path.join(carpeta, archivo)
                #print(f"Encontrado archivo: {ruta_completa}")  # Depuración
                try:
                    # Leer el archivo CSV
                    df = pd.read_csv(ruta_completa, sep=';', decimal=',')
                    # Convierto la columna de fechas que viene como texto a formato datatime de pandas
                    df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce', utc=True)
                    dataframes.append(df)
                except Exception as e:
                    print(f"Error al leer el archivo {ruta_completa}: {e}")
    
    # Concatenar todos los DataFrames si hay datos
    if dataframes:       
        df_combinado = pd.concat(dataframes, ignore_index=True)
        # Información básica
        print ("Tipo datos")
        print (df_combinado.info())
        contar_valores_distintos = df_combinado.nunique() # Muestra cantidad valores unicos de cada señal
        print ("Valores distintos por columnas:")
        print (contar_valores_distintos)
        return df_combinado
    else:
        print("No se encontraron archivos CSV en las subcarpetas.")
        return pd.DataFrame()  # Devuelve un DataFrame vacío si no hay datos
    
# Funcion para convertir tipo datos
def convertir_columnas_a_numerico(df, columnas, tipo_dato = float):
    """
    Convierte las columnas especificadas a datos numéricos, eliminando las filas
    con valores no numéricos en esas columnas.

    Parámetros:
        df (pd.DataFrame): El dataframe a procesar.
        columnas (list): Lista de nombres de columnas a convertir.

    Retorna:
        pd.DataFrame: El dataframe con las columnas convertidas y filas no numéricas eliminadas.
    """
    # Aplicar la conversión a numérico con manejo de errores en las columnas especificadas
    for columna in columnas:
        df[columna] = pd.to_numeric(df[columna], errors='coerce')

    # Eliminar filas con NaN en las columnas seleccionadas
    df = df.dropna(subset=columnas)

    # Convertir los datos al tipo especificado
    df.loc[:, columnas] = df[columnas].astype(tipo_dato)

    return df

def EDA (df):
    # Análsis exploratio de datos
    sns.histplot(df)
    plt.show()

In [10]:
# Ruta de la carpeta base donde estan los ficheros CSV(cambiar por la tuya)
ruta_base = 'datos_PACS'

# Llamar a la función para crear el dataframe completo
df_leido = leer_csv_en_carpetas(ruta_base)

# Convierto a tipo numerico las columnas que necesito
# En los datos vendrán valores tipo texto "BAD", "Error", ... que provienen de PI
columnas_a_convertir = ['SAB:CBOP.A81.PAC11.AP001XH01']
tipo_conversion = int
df_leido = convertir_columnas_a_numerico (df_leido, columnas_a_convertir, tipo_conversion)
columnas_a_convertir = ['SAB:CBOP.A81.PAC13.AP001XH01']
tipo_conversion = int
df_leido = convertir_columnas_a_numerico (df_leido, columnas_a_convertir, tipo_conversion)

Tipo datos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5830704 entries, 0 to 5830703
Data columns (total 3 columns):
 #   Column                        Dtype              
---  ------                        -----              
 0   Timestamp                     datetime64[ns, UTC]
 1   SAB:CBOP.A81.PAC11.AP001XH01  object             
 2   SAB:CBOP.A81.PAC13.AP001XH01  object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 133.5+ MB
None
Valores distintos por columnas:
Timestamp                       5830571
SAB:CBOP.A81.PAC11.AP001XH01         12
SAB:CBOP.A81.PAC13.AP001XH01         12
dtype: int64


In [ ]:
# Análisis exploratorio de datos
EDA (df_leido)

# Contador horas y números arranque

In [11]:
# Parámetros
# dataframe: dataframe a mirar
# tag: nombre señal a mirar
# estado_marcha: se indica si marcha es un 1 o un 0
# intervalo: frecuencia de muestreo

def contar_horas_arranques(dataframe, tag, estado_marcha, intervalo):
    # Asegurarse de que la columna existe
    if tag not in dataframe.columns:
        raise ValueError("El DataFrame no contiene la señal: " + tag)
    
    # Calcular la suma total
    suma_total = dataframe[tag].sum()
    
    # Contar cambios de señal de 0 a 1
    # Alternative longer form
    if estado_marcha == 1:
        estado_marcha = 1
        estado_paro = 0 
    else:   
        estado_marcha = 0
        estado_paro = 1

    cambios_estado = ((dataframe[tag] == estado_marcha) & (dataframe[tag].shift(1) == estado_paro)).sum()
    
    segundos_totales = suma_total * int(intervalo)
    horas = segundos_totales // 3600
    minutos = (segundos_totales % 3600) // 60
    return {
        'horas_arrancado': horas,
        'minutos_arrancado': minutos,
        'arranques': cambios_estado
    }


In [13]:
# Ejemplo de uso
# Primero hago una copia
df_tratado = df_leido.copy()

tag = "SAB:CBOP.A81.PAC11.AP001XH01"
estado_marcha = 1
intervalo_muestreo_datos = 60 # Ponerlo en segundos
resultado = contar_horas_arranques(df_leido, tag, estado_marcha, intervalo_muestreo_datos)

print("Número arranques:", resultado['arranques'])
# El tiempo arrancado lo 
print(f"Tiempo arrancado de {tag}: {resultado['horas_arrancado']} horas y {resultado['minutos_arrancado']} minutos")

Número arranques: 715
Tiempo arrancado de SAB:CBOP.A81.PAC11.AP001XH01: 46037.0 horas y 19.0 minutos


# Máximos y mínimos

In [8]:
import pandas as pd
from tqdm.notebook import tqdm
import PIconnect as PI
from PIconnect.PIConsts import SummaryType
from datetime import datetime, timedelta

def leer_excel_a_dataframe(ruta_excel):
    try:
        df = pd.read_excel(ruta_excel)
        return df
    except Exception as e:
        print(f"Error al leer el archivo {ruta_excel}: {e}")
        return pd.DataFrame()

fichero = "señales_PI_SABON.xlsx"
print ("Leyendo fichero TAGS")
tags_SABON = leer_excel_a_dataframe(fichero)
cantidad_tags_a_leer = 10  # Este parámetro es para que lea solo los n primeros datos de la tabla de tags

fecha_inicio = datetime(2024, 1, 1, 0, 0, 0)  # Cambia por la fecha de inicio deseada
fecha_fin = datetime(2025, 2, 1, 0, 0, 0)  # Cambia por el rango de tiempo deseado

with PI.PIServer() as server:
    resultados = []
    # Agregamos tqdm para mostrar el progreso
    for index, row in tqdm(tags_SABON.head(cantidad_tags_a_leer).iterrows(), 
                           total=cantidad_tags_a_leer, desc="Procesando tags", leave=True):
        tag_name = row['Name']
        tipo = row['pointtype']
        print(f"Procesando: {tag_name} - {tipo}")
        if tipo in ['Float64', 'Int16', 'Int32', 'Float32']:
            try:
                points = server.search(tag_name)[0]
                fecha_actual = fecha_inicio
                # Agregamos tqdm para mostrar el progreso mensual
                total_meses = ((fecha_fin.year - fecha_inicio.year) * 12 + fecha_fin.month - fecha_inicio.month) + 1
                with tqdm(total=total_meses-1, desc=f"Procesando {tag_name}", leave=False) as pbar:
                    while fecha_actual < fecha_fin:
                        mes_siguiente = (fecha_actual.replace(day=28) + timedelta(days=4)).replace(day=1)
                        fecha_siguiente = min(mes_siguiente, fecha_fin)
                        
                        data = points.summary(fecha_actual, fecha_siguiente, SummaryType.MAXIMUM | SummaryType.MINIMUM)
                        data['Tag'] = tag_name
                        resultados.append(data)
                        
                        fecha_actual = fecha_siguiente
                        pbar.update(1)
            except Exception as e:
                print(f"Error al obtener datos para {tag_name}: {e}")

    df_resultados = pd.concat(resultados)

    # Unificar los resultados por Tag
    df_resultados_unificado = df_resultados.groupby('Tag').agg(
        MINIMUM=('MINIMUM', 'min'),
        MAXIMUM=('MAXIMUM', 'max'),
        TIMESTAMP_MIN=('MINIMUM', 'idxmin'),
        TIMESTAMP_MAX=('MAXIMUM', 'idxmax')
    ).reset_index()

    print("Finalizado")
    df_resultados_unificado.to_csv("tags_minimos_maximos.csv", sep=";", decimal=",", encoding="utf-8-sig")


Leyendo fichero TAGS


Procesando tags:   0%|          | 0/10 [00:00<?, ?it/s]

Procesando: Sabón.Potencia Activa - Float64


Procesando Sabón.Potencia Activa:   0%|          | 0/13 [00:00<?, ?it/s]

Procesando: SAB.Calidad_ABC - Float64


...

Procesando: SAB.ERM_FC1.Descripcion estado - String
Procesando: SAB.ERM_FC1.Down - Int16


Procesando SAB.ERM_FC1.Down:   0%|          | 0/13 [00:00<?, ?it/s]

Procesando: SAB.ERM_FC1.Fail - Int16


Procesando SAB.ERM_FC1.Fail:   0%|          | 0/13 [00:00<?, ?it/s]

Finalizado
